<center>
<h2>Syntactic Parsing part 1: <br> Graph-based Dependency parsing</h2>
<p style="text-align:center">
Natural Language Processing<br>
(COM4513/6513)<br>
<br>
<a href="http://andreasvlachos.github.io">Andreas Vlachos</a><br>
a.vlachos@sheffield.ac.uk<br>
<small>Department of Computer Science<br>
University of Sheffield
</small>
</p>
</center>

### So far: classification

Given an instance $x$ (e.g. document), predict a label $y \in \cal Y$

Tasks:
- sentiment analysis
- topic classification
- etc.

Algorithms:
- perceptron
- logistic regression

### So far: sequence labeling

Given a sequence of words $\mathbf{x} = [x_1,... x_N]$ predict a sequence of labels $\mathbf{y} \in \cal Y^N$

Tasks:
- part of speech tagging
- named entity recognition
- etc.

Algorithms:
- structured perceptron
- hidden Markov models
- conditional random fields

### In this lecture

Syntactic parsing 1: Graph-based Dependency parsing
<img src="images/tikz/test.png" style="width:1000px; border:none; box-shadow:none;">

- Definitions
- Graph-based parsing

### Problem setup

Training data is pairs of word sequences (sentences) and dependency trees:

\begin{align}
D_{train} & = \{(\mathbf{x}^1,G_x^1)...(\mathbf{x}^M,G_x^M)\} \\
\mathbf{x}^m & = [x_1,... x_N]\\
graph\; G_\mathbf{x} &= (V_\mathbf{x}, A_\mathbf{x})\\
vertices\; V_\mathbf{x} &=\{0,1,...,N\}\\
edges\; A_\mathbf{x} &=\{(i,j,k)|i,j\in V, k \in L \text{(labels)}\}
\end{align}

We want to learn a model to predict the best graph:

$$
\hat G_\mathbf{x} = \mathop{\arg \max}\limits_{G_\mathbf{x} \in \cal G_\mathbf{x}} score(G_\mathbf{x},\mathbf{x})
$$

### Some constraints

**Connected**: every word can be reached from any other word ignoring edge directionality

**Acyclic**: can't re-visit the same word on a directed path

**Single-Head**: every word can have only one head

### Well-formed dependency tree?

<img src="images/tikz/depParse.png" style="border:none; box-shadow:none;">

- Connected? <span class="fragment">NO</span>
- Acyclic? <span class="fragment">YES</span>
- Single-headed? <span class="fragment">YES</span>

### Well-formed dependency tree

<img src="images/tikz/depParseRoot.png" style="border:none; box-shadow:none;">

Add a special root node with edges to any nodes without heads (main verb and punctuation).

### Learning a dependency parser

We want to learn a model to predict the best graph:

$$
\hat G_\mathbf{x} = \mathop{\arg \max}\limits_{G_\mathbf{x} \in \cal G_\mathbf{x}} score(G_\mathbf{x},\mathbf{x})
$$
where the $G_x$ is a well-formed dependency tree.

Can we learn it using what we know?

Enumeration over all possible graphs will be expensive.

How about a classifier that predicts each edge?

Maybe. But predicting an edge makes some edges invalid due to the acyclic and single-head constraints.		

### Maximum Spanning Tree

Score all edges, but keep only the max spanning tree.
<img src="images/MSTParsing.png" style="border:none; box-shadow:none;">

Exact solution in $O(N^2)$ time using Chu-Liu-Edmonds, a modification to Kruskal for directed graphs.

### Kruskal's algorithm


<p><img src="images/MST_kruskal_en.gif" style="float: left; width:45%; border:none; box-shadow:none;">

<p style="float: right;  font-size: 100%; border:3px; width: 50%; border-radius: 25px; background-color:lightgrey; border-style:solid; border-color:black; padding: 0.3em;">
<b>Input</b>: scored edges $E$<br>
sort $E$ by cost(opposit of score)<br>
<b>while</b> $G$ not spanning:<br>
&nbsp;&nbsp;&nbsp; pop the next edge $e$<br>
&nbsp;&nbsp;&nbsp; <b>if</b> connecting different trees:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  add $e$ to $G$
</p>
</p>

<p style="text-align: left; font-size:60%">By Schulllz (Own work) [<a href="http://creativecommons.org/licenses/by-sa/3.0">CC BY-SA 3.0</a>], <a href="https://commons.wikimedia.org/wiki/File%3AMST_kruskal_en.gif">via Wikimedia Commons</a></p>

### Graph-based parsing

Decompose the graph score into arc scores:

\begin{align}
				\hat G_\mathbf{x} & = \mathop{\arg \max}\limits_{G_\mathbf{x} \in \cal G_\mathbf{x}} score(G_\mathbf{x},\mathbf{x})\\
				 & = \mathop{\arg \max}\limits_{G_\mathbf{x} \in \cal G_\mathbf{x}} \mathbf{w} \cdot \Phi(G_\mathbf{x},\mathbf{x}) \quad \text{(linear model)}\\
				 & = \mathop{\arg \max}\limits_{G_\mathbf{x} \in \cal G_\mathbf{x}} \sum_{(i,j,l) \in A_x} \mathbf{w} \cdot \phi((i,j,l),\mathbf{x}) \quad \text{(arc-factored)}
				 \end{align}

Can learn the weights with the structured perceptron!

### Feature extraction

What should $\phi((head,dependent,label),\mathbf{x})$ be? Discuss!

<img src="images/tikz/depParseScope.png" style="width:1000px; border:none; box-shadow:none;">

- unigram: head=had, head=had&label=dobj, head=VERB
- bigram: head=had&dependent=effect
- head=VERB&dependent=NOUN&between=ADJ
- head=had&label=dobj&other-label=nsubj <span style="color:red" class="fragment">NO!!! Breaks the arc-factored scoring</span>

### More Global models

Even though inference and learning are global, features are localized to arcs.

Can we have more global features?

Yes we can! Consider subgraphs spanning a few edges. But inference becomes harder, requiring more complex dynamic programs and clever approximations.

Is it worth it? 

Syntactic parsing is used everywhere, thus better compromises between speed and accuracy are always welcome!

### Evaluation

<img src="images/tikz/depParse.png" style="border:none; box-shadow:none;">

Head-finding word-accuracy:
- unlabeled: % of words with the right head
- labeled: % of words with the right head and label

Sentence accuracy: % of sentences with correct graph

### Bibliography

- Nivre and McDonald's tutorial [slides](http://stp.lingfil.uu.se/~nivre/eacl14.html)
- McDonald's [paper](http://www.ryanmcd.com/papers/MS-CIS-05-11.pdf) on graph-based parsing
- Kruskal's [algorithm](https://en.wikipedia.org/wiki/Kruskal%27s_algorithm) and Marina Valeeva's [visualization](http://www.coli.uni-saarland.de/~yzhang/rapt-ws1213/slides/valeeva.pdf) of Chu-Liu-Edmonds

### Coming up next

Transition-based dependency parsing

A.k.a. how to use a (small) classifier to predict a graph!